In [38]:
import os
import pandas as pd
import json
import os, sys
print(os.path.join(os.path.dirname(os.getcwd()), "app"))

sys.path.append("/Users/officialbiznas/Documents/GitHub/stockFries/app")
import scraper
from config import fund_dict, figi_api_key, scrapped_json_fn
import requests
from bs4 import BeautifulSoup

/Users/officialbiznas/Documents/GitHub/stockFries/app


In [12]:
#04878Q863
def googleSearch(query):
    with requests.session() as c:
        url = 'https://finance.yahoo.com/'
        query = {'q': query}
        urllink = requests.get(url, params=query)
        print(urllink.url)

googleSearch('04878Q863')

https://finance.yahoo.com/?q=04878Q863


## Testing Figi

In [168]:
#https://www.pythonsherpa.com/tutorials/1/
jobs = [
    {'idType': 'ID_CUSIP', 'idValue': '04878Q863', 'micCode': 'XETR'}
]
job={'idType': 'ID_CUSIP', 'idValue': '04878Q863', 'micCode': 'XETR'}
jobs = [
    {'idType': 'TICKER', 'idValue': 'ADS', 'micCode': 'XETR'},
    {'idType': 'TICKER', 'idValue': 'BAS', 'micCode': 'XETR'},    
    {'idType': 'TICKER', 'idValue': 'DTE', 'micCode': 'XETR'},
    {'idType': 'TICKER', 'idValue': 'SAPdafwef', 'micCode': 'XETR'},
    {'idType': 'TICKER', 'idValue': 'SIE', 'micCode': 'XETR'},
    {'idType':'ID_CUSIP','idValue':'04878Q863','micCode': 'XETR'}
]

job = [{'idType':'ID_CUSIP','idValue':'04878Q863'},
      {'idType':'ID_CUSIP','idValue':'07987C204'},
      {'idType':'ID_CUSIP','idValue':'awefefdsfaewc'}]

In [179]:
openfigi_apikey = '34dada0c-5a22-4f5a-a095-739afe8fe591'  # Please put your own API Key here
openfigi_url = 'https://api.openfigi.com/v2/mapping'
openfigi_headers = {'Content-Type': 'text/json'}

def map_jobs(jobs_):
    if openfigi_apikey:
        openfigi_headers['X-OPENFIGI-APIKEY'] = openfigi_apikey
    response = requests.post(url=openfigi_url, headers=openfigi_headers,
                             json=jobs_)
    print(response)
    respon_json = response.json()
    #if response.status_code != 200:
        #raise Exception()#Exception('Bad response code {}'.format(str(response.status_code)))
    return respon_json

In [180]:
map_jobs(job)

<Response [200]>


[{'data': [{'figi': 'BBG000P3P4D5',
    'name': 'ATLANTIC POWER CORP',
    'ticker': 'ATP',
    'exchCode': 'CN',
    'compositeFIGI': 'BBG000P3P4D5',
    'uniqueID': 'EQ0000000001461369',
    'securityType': 'Common Stock',
    'marketSector': 'Equity',
    'shareClassFIGI': 'BBG001SL3NS6',
    'uniqueIDFutOpt': None,
    'securityType2': 'Common Stock',
    'securityDescription': 'ATP'},
   {'figi': 'BBG000P3P5J6',
    'name': 'ATLANTIC POWER CORP',
    'ticker': 'ATP',
    'exchCode': 'CT',
    'compositeFIGI': 'BBG000P3P4D5',
    'uniqueID': 'EQ0000000001461369',
    'securityType': 'Common Stock',
    'marketSector': 'Equity',
    'shareClassFIGI': 'BBG001SL3NS6',
    'uniqueIDFutOpt': None,
    'securityType2': 'Common Stock',
    'securityDescription': 'ATP'},
   {'figi': 'BBG000P3P999',
    'name': 'ATLANTIC POWER CORP',
    'ticker': 'ATP',
    'exchCode': 'CJ',
    'compositeFIGI': 'BBG000P3P4D5',
    'uniqueID': 'EQ0000000001461369',
    'securityType': 'Common Stock',
    '

In [181]:
map_jobs(job)[1]['data'][0]['ticker']

<Response [200]>


'BLU'

In [182]:
map_jobs(job)[0]['data'][0]['ticker']

<Response [200]>


'ATP'

## Writing Functions

In [2]:
'''fn = os.path.join(os.path.dirname(os.getcwd()), 'app', 'json_df.txt') #app/json_df.txt
print(fn)
with open(fn, 'r') as f:
        json_df = json.load(f)
df = pd.read_json(json_df, orient='split').rename_axis('cusip').reset_index()#
list_of_cusips = df.cusip.tolist()
#print(list_of_cusips)
#df.tail()'''

/Users/officialbiznas/Documents/GitHub/stockFries/app/json_df.txt


In [42]:
import pandas as pd
import os, sys, json
#print(os.path.join(os.path.dirname(os.getcwd()), "app"))
#sys.path.append("/Users/officialbiznas/Documents/GitHub/stockFries/app")
#import scraper
from config import fund_dict, figi_api_key
import requests


ImportError: cannot import name 'figi_api_key' from 'config' (/Users/officialbiznas/Documents/GitHub/stockFries/app/config.py)

In [44]:
def get_tickers_from_cusips(list_of_cusips_):
    '''
    input: list_of_cusips
    output: list_of_stock_tickers
    '''
    #jobs_ = [{'idType':'ID_CUSIP','idValue':cusip} for cusip in list_of_cusips_]
    list_of_tickers = []
    openfigi_apikey = figi_api_key 
    openfigi_url = 'https://api.openfigi.com/v2/mapping'
    openfigi_headers = {'Content-Type': 'text/json'}
    if openfigi_apikey:
        openfigi_headers['X-OPENFIGI-APIKEY'] = openfigi_apikey
    for cusip in list_of_cusips_:
        job_ = [{'idType':'ID_CUSIP','idValue':cusip}]
        try:
            response = requests.post(url=openfigi_url, headers=openfigi_headers,
                         json=job_)
            respon_json = response.json()
            response_ticker = respon_json[0]['data'][0]['ticker']
            list_of_tickers.append(response_ticker)
            #print(response_ticker)
        except:
            list_of_tickers.append(None)
    return list_of_tickers

def generate_yahoo_link(ticker_):
    return 'https://finance.yahoo.com/quote/{}?p={}'.format(str(ticker_),str(ticker_))

def generate_and_save_tiker_yahoo_json(fn_):
    '''
    input: fn; absolute paht
    output: json df with stocker ticker and 
        yahoo url colums added and re-saved
    '''
    with open(fn_, 'r') as f:
        json_df = json.load(f)
    df_ = pd.read_json(json_df, orient='split').rename_axis('cusip').reset_index()#
    list_of_cusips = df.cusip.tolist()
    df_['ticker'] = get_tickers_from_cusips(list_of_cusips)
    df_['URL_Yahoo'] = df_.ticker.apply(generate_yahoo_link)
    json_df_ = df_.to_json(orient='split')
    with open(fn_, 'w') as f:
        json.dump(json_df_, f)
    


In [ ]:
%%time
fn = os.path.join(os.path.dirname(os.getcwd()), 'app', 'json_df.txt')
generate_and_save_tiker_yahoo_json(fn)


In [29]:
%%time
output = get_tickers_from_cusips(list_of_cusips)
print(output)
#df['tickers']=output
#df.tail()

TSLA
BA
NFLX
AAPL
XOM
NEM
AMGN
MO
INTC
SQ
BABA
FB
XOP
SPY
WFC
GOLD
ABBV
DYH
PG
JPM
BAC
XLE
NKE
COST
T
MA
KO
SHOP
LMT
MU
QCOM
NVDA
FDX
BMY
WDC
CHTR
VIAC
DIS
HUM
TRVC
EBA
RTX
SPGI
COF
NEE
SNAP
MSFT
UPS
NOW
CMG
CSX
GM
HAL
CME
BIDUN
V
AFW
JNJ
CAH
EW
LOW
VEEV
CVS
WDAY
CF
AWM
BIIB
BUD
KHC
LULU
CAT
AIG
OXY
MLM
IBM
NSC
HD
DLTR
SPG
WBA
DOCU
AIY
KSU
PYPL
GOOGL
XLNX
ORCL
FNV
MPC
NRA
BRK/B
SLB
DG
XBI
TDOC
BX
GS
SPLK
HIG
COP
AMD
LLY
EFA
BP
4I1
XLF
TSN
BBY
ADBE
UNH
TXN
FTNT
PH
MMM
ADSK
YUM
TSM
UNP
EQR
EWZ
ETSY
IBB
SBAC
PFE
RDS/A
GDXJ
ISRG
VLO
SU
ABMD
AFL
W3U
VRSN
SE
USB
ALS
PNC
IWM
AMT
CP
TAL
VEN
BHC
PEP
K
SGENEUR
SYF
IEP
SO
SRE
TTWO
ANTM
MOS
REGN
ALSN
PSX
MRK
CMA
CLX
VMW
XLK
CRM
DWD
NTAP
KRE
CTXS
EA
HLT
ICE
SBUX
8CW
DFS
TWTR
DHI
GDX
ALB
AWI
PHM
CB1A
TWLO
IRBT
LRCX
CPRT
NVSN
FL
URI
CHRW
SYY
TER
ANET
DISH
XLU
AEP
CNR
SEDG
LBRDK
ALXN
PRU
MEDP
RS
ORLY
CDNS
HAS
CMI
AMZN
AXP
TSCO
NLOK
KR
JD
WPM
VRTX
COG
KMI
DGE$
HSY
RCII
VIPS
QRVO
EFX
CHE
CHV
EMR
MCD
GOOG
KL
CNC
MAR
KMB
AVB
UN
CVNA
INTU
TTD
MBT
LSCC
TOL


In [31]:
output

['TSLA',
 'BA',
 'NFLX',
 'AAPL',
 'XOM',
 'NEM',
 'AMGN',
 'MO',
 'INTC',
 'SQ',
 'BABA',
 'FB',
 'XOP',
 'SPY',
 'WFC',
 'GOLD',
 'ABBV',
 'DYH',
 'PG',
 'JPM',
 'BAC',
 'XLE',
 'NKE',
 'COST',
 'T',
 'MA',
 'KO',
 'SHOP',
 'LMT',
 'MU',
 'QCOM',
 'NVDA',
 'FDX',
 'BMY',
 'WDC',
 'CHTR',
 'VIAC',
 'DIS',
 'HUM',
 'TRVC',
 'EBA',
 'RTX',
 'SPGI',
 'COF',
 'NEE',
 'SNAP',
 'MSFT',
 'UPS',
 'NOW',
 'CMG',
 'CSX',
 'GM',
 'HAL',
 'CME',
 'BIDUN',
 'V',
 'AFW',
 None,
 'JNJ',
 'CAH',
 'EW',
 'LOW',
 'VEEV',
 'CVS',
 'WDAY',
 'CF',
 'AWM',
 'BIIB',
 'BUD',
 'KHC',
 None,
 'LULU',
 'CAT',
 'AIG',
 'OXY',
 'MLM',
 'IBM',
 'NSC',
 'HD',
 'DLTR',
 'SPG',
 'WBA',
 'DOCU',
 'AIY',
 'KSU',
 'PYPL',
 'GOOGL',
 'XLNX',
 'ORCL',
 'FNV',
 'MPC',
 'NRA',
 'BRK/B',
 'SLB',
 'DG',
 'XBI',
 'TDOC',
 'BX',
 'GS',
 'SPLK',
 'HIG',
 'COP',
 'AMD',
 'LLY',
 'EFA',
 'BP',
 '4I1',
 'XLF',
 'TSN',
 'BBY',
 'ADBE',
 'UNH',
 'TXN',
 'FTNT',
 'PH',
 'MMM',
 'ADSK',
 'YUM',
 'TSM',
 'UNP',
 'EQR',
 'EWZ',
 'ETSY',


In [32]:
df['ticker'] = output

In [33]:
df['URL_Yahoo'] = df.ticker.apply(generate_yahoo_link)

In [35]:
df[df.ticker.notnull()].tail()

,cusip,Company,Value_x,Shares_x,Ownership,Value_y,Shares_y,Change,ticker,URL_Yahoo
2547,98850P109,Yum China Holdings Inc,NaN,NaN,NaN,3475.0,81519.0,NaN,YUMC,https://finance.yahoo.com/quote/YUMC?p=YUMC
2548,98885E103,Zafgen Inc,NaN,NaN,NaN,1357.0,1762826.0,NaN,ZFGNUSD,https://finance.yahoo.com/quote/ZFGNUSD?p=ZFGNUSD
2549,989207105,Zebra Technologies Corp,NaN,NaN,NaN,1227.0,6686.0,NaN,ZBRA,https://finance.yahoo.com/quote/ZBRA?p=ZBRA
2550,989701107,Zions Bancorp Na,NaN,NaN,NaN,2724.0,101791.0,NaN,ZION,https://finance.yahoo.com/quote/ZION?p=ZION
2551,98985W102,Zymeworks Inc,NaN,NaN,NaN,61114.0,1722963.0,NaN,ZYME,https://finance.yahoo.com/quote/ZYME?p=ZYME


## Testing Scripts

In [1]:
import os, sys
print(os.path.join(os.path.dirname(os.getcwd()), "app"))
sys.path.append("/Users/officialbiznas/Documents/GitHub/stockFries/app")
import scraper
import config
#from config import scrapped_json_fn_no_ticker

/Users/officialbiznas/Documents/GitHub/stockFries/app


In [10]:
%%time
scraper.pandas_analysis(config.fund_dict)

['https://www.sec.gov/Archives/edgar/data/1535392/000156761920015183/xslForm13F_X01/form13fInfoTable.xml', 'https://www.sec.gov/Archives/edgar/data/1535392/000156761920010282/xslForm13F_X01/form13fInfoTable.xml']
['https://www.sec.gov/Archives/edgar/data/1544676/000154467620000008/xslForm13F_X01/13f.xml', 'https://www.sec.gov/Archives/edgar/data/1544676/000154467620000006/xslForm13F_X01/13fv2.xml']
['https://www.sec.gov/Archives/edgar/data/1425649/000110465920095369/xslForm13F_X01/infotable.xml', 'https://www.sec.gov/Archives/edgar/data/1425649/000110465920062428/xslForm13F_X01/infotable.xml']
[]
['https://www.sec.gov/Archives/edgar/data/1104186/000110418620000005/xslForm13F_X01/t13f-0620.xml', 'https://www.sec.gov/Archives/edgar/data/1104186/000110418620000004/xslForm13F_X01/t13f-0320.xml']
['https://www.sec.gov/Archives/edgar/data/1226355/000122635519000012/xslForm13F_X01/Harvest13f_3q2019.inftab.xml', 'https://www.sec.gov/Archives/edgar/data/1226355/000122635519000009/xslForm13F_X01

In [2]:
%%time
scraper.generate_and_save_tiker_yahoo_json(config.scrapped_json_fn_no_ticker)

CPU times: user 1min 4s, sys: 4.44 s, total: 1min 9s
Wall time: 19min 33s


In [198]:
def get_ticker_from_cusip(cusip_):
    '''
    input: list_of_cusips
    output: list_of_stock_tickers
    '''
    #jobs_ = [{'idType':'ID_CUSIP','idValue':cusip} for cusip in list_of_cusips_]
    list_of_tickers = []
    if openfigi_apikey:
        openfigi_headers['X-OPENFIGI-APIKEY'] = openfigi_apikey
    
    job_ = [{'idType':'ID_CUSIP','idValue':cusip_}]

    response = requests.post(url=openfigi_url, headers=openfigi_headers,
                             json=job_)
    #print(response)
    respon_json = response.json()
    try:
        return respon_json['data'][0]['ticker']
    except:
        return None
    


In [ ]:
%%time
#df['tickers2'] = df.cusip.apply(get_ticker_from_cusip)

In [199]:
df.tail()

,cusip,Company,Value_x,Shares_x,Ownership,Value_y,Shares_y,Change,tickers,tickers2
2612,N6596X109,Nxp Semiconductors Nv,NaN,NaN,NaN,2128.0,25663.0,NaN,<generator object get_tickers_from_cusips.<loc...,None
2613,N70544106,Playa Hotels & Resorts Nv,NaN,NaN,NaN,4103.0,2344296.0,NaN,<generator object get_tickers_from_cusips.<loc...,None
2614,Y2573F102,Flex Ltd,NaN,NaN,NaN,98359.0,11744352.0,NaN,<generator object get_tickers_from_cusips.<loc...,None
2615,Y41053102,International Seaways Inc,NaN,NaN,NaN,21923.0,917646.0,NaN,<generator object get_tickers_from_cusips.<loc...,None
2616,Y75638109,Atlas Corp,NaN,NaN,NaN,1423.0,185009.0,NaN,<generator object get_tickers_from_cusips.<loc...,None


In [200]:
df[df.tickers2.notnull()]

,cusip,Company,Value_x,Shares_x,Ownership,Value_y,Shares_y,Change,tickers,tickers2


## Functions take 2

In [22]:
def get_tickers_from_cusips_one_shot(list_of_cusips_):
    '''
    input: list_of_cusips
    output: list_of_stock_tickers
    '''
    jobs_ = [{'idType':'ID_CUSIP','idValue':cusip} for cusip in list_of_cusips_]
    print(jobs_)
    if openfigi_apikey:
        openfigi_headers['X-OPENFIGI-APIKEY'] = openfigi_apikey
    response = requests.post(url=openfigi_url, headers=openfigi_headers,
                             json=jobs_)
    #print(response.content)
    #respon_json = response.json()
    if response.status_code != 200:
        raise Exception()#Exception('Bad response code {}'.format(str(response.status_code)))
    return response

    '''#print(jobs)
    response_data_ = map_jobs(jobs_)
    list_of_tickers = []
    for response_cusip in response_data_:
        print(response_cusip)
        try:
            list_of_tickers.append(response_cusip['data'][0]['ticker'] for x in response_data)
        except:
            list_of_tickers.append(None)
    return list_of_tickers'''

openfigi_apikey = '34dada0c-5a22-4f5a-a095-739afe8fe591'  # Please put your own API Key here
openfigi_url = 'https://api.openfigi.com/v2/mapping'
openfigi_headers = {'Content-Type': 'text/json'}

def map_jobs(jobs_):
    if openfigi_apikey:
        openfigi_headers['X-OPENFIGI-APIKEY'] = openfigi_apikey
    response = requests.post(url=openfigi_url, headers=openfigi_headers,
                             json=jobs_)
    print(response)
    respon_json = response.json()
    #if response.status_code != 200:
        #raise Exception()#Exception('Bad response code {}'.format(str(response.status_code)))
    return respon_json

In [23]:
fn = os.path.join(os.path.dirname(os.getcwd()), 'app', 'json_df.txt') #app/json_df.txt
print(fn)
with open(fn, 'r') as f:
        json_df = json.load(f)
df = pd.read_json(json_df, orient='split').rename_axis('cusip').reset_index()#
list_of_cusips = df.cusip.tolist()
print(list_of_cusips[-10:])
#df.tail()

/Users/officialbiznas/Documents/GitHub/stockFries/app/json_df.txt
['M96088105', 'M98068105', 'N07059210', 'N20944109', 'N53745100', 'N6596X109', 'N70544106', 'Y2573F102', 'Y41053102', 'Y75638109']


In [24]:
%%time
tickers_figi = get_tickers_from_cusips_one_shot(list_of_cusips)

[{'idType': 'ID_CUSIP', 'idValue': '88160R101'}, {'idType': 'ID_CUSIP', 'idValue': '097023105'}, {'idType': 'ID_CUSIP', 'idValue': '64110L106'}, {'idType': 'ID_CUSIP', 'idValue': '037833100'}, {'idType': 'ID_CUSIP', 'idValue': '30231G102'}, {'idType': 'ID_CUSIP', 'idValue': '651639106'}, {'idType': 'ID_CUSIP', 'idValue': '031162100'}, {'idType': 'ID_CUSIP', 'idValue': '02209S103'}, {'idType': 'ID_CUSIP', 'idValue': '458140100'}, {'idType': 'ID_CUSIP', 'idValue': '852234103'}, {'idType': 'ID_CUSIP', 'idValue': '01609W102'}, {'idType': 'ID_CUSIP', 'idValue': '30303M102'}, {'idType': 'ID_CUSIP', 'idValue': '78468R556'}, {'idType': 'ID_CUSIP', 'idValue': '78462F103'}, {'idType': 'ID_CUSIP', 'idValue': '949746101'}, {'idType': 'ID_CUSIP', 'idValue': '067901108'}, {'idType': 'ID_CUSIP', 'idValue': '00287Y109'}, {'idType': 'ID_CUSIP', 'idValue': '87612E106'}, {'idType': 'ID_CUSIP', 'idValue': '742718109'}, {'idType': 'ID_CUSIP', 'idValue': '46625H100'}, {'idType': 'ID_CUSIP', 'idValue': '0605

Exception: 

In [15]:
tickers_figi

<Response [400]>

In [138]:
jobs = [{'idType':'ID_CUSIP','idValue':cusip} for cusip in list_of_cusips]
response_data = map_jobs(jobs)
tickers = [x['data'][0]['ticker'] for x in response_data]
df['tickers']=tickers
df.tail()

Exception: Bad response code 400

In [137]:
tickers

['ATP', 'BLU']

In [6]:


fn = os.path.join(os.path.dirname(os.getcwd()), 'app', 'json_df.txt') #app/json_df.txt
with open(fn, 'r') as f:
        json_df = json.load(f)

print(fn)
df = pd.read_json(json_df, orient='split')
ownership = df[['Company', 'Ownership']
               ].dropna().head(n=20).values.tolist()
positives = df.sort_values(
    'Change', ascending=False)[['Company', 'Change']].dropna().head(n=20).values.tolist()
negatives = df.sort_values(
    'Change')[['Company', 'Change']].dropna().head(n=20).values.tolist()
df.tail()

/Users/officialbiznas/Documents/GitHub/stockFries/app/json_df.txt


,index,Company,Value_x,Shares_x,Ownership,Value_y,Shares_y,Change
2612,N6596X109,Nxp Semiconductors Nv,NaN,NaN,NaN,2128.0,25663.0,NaN
2613,N70544106,Playa Hotels & Resorts Nv,NaN,NaN,NaN,4103.0,2344296.0,NaN
2614,Y2573F102,Flex Ltd,NaN,NaN,NaN,98359.0,11744352.0,NaN
2615,Y41053102,International Seaways Inc,NaN,NaN,NaN,21923.0,917646.0,NaN
2616,Y75638109,Atlas Corp,NaN,NaN,NaN,1423.0,185009.0,NaN


In [17]:
df.head()

,index,Company,Shares_x,Value_x,Ownership,Shares_y,Value_y,Change
0,88160R,Tesla Inc,42116277.0,192177977.0,2.94,NaN,NaN,NaN
1,097023,Boeing Co,3658828.0,167087119.0,2.56,394102.0,58777.0,89.23
2,64110L,Netflix Inc,5576767.0,151505143.0,2.32,9129848.0,3428260.0,-63.71
3,037833,Apple Inc,375380.0,136938624.0,2.09,347481.0,88361.0,7.43
4,30231G,Exxon Mobil Corp,3060000.0,136843200.0,2.09,3707932.0,140056.0,-21.17
